# Text Emotion Classification using Machine Learning

In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [5]:
df = pd.read_csv('data/train.txt', sep = ';')
df.head()

,i didnt feel humiliated,sadness
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [6]:
# Since there is no header, now I will add header to the data.
df.columns = ['Text', 'Emotions']
df.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


### Since we are using texts, it is best to tokenize the data.

In [7]:
texts = df['Text'].tolist()
labels = df['Emotions'].tolist()

#Tokenizing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

### All sentence must be of same length, so we now add sentence padding

In [8]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [9]:
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

### One Hot Encoding to vectorizing the data to Zeros and One.

In [10]:
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

## Model Creation

In [12]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=len(one_hot_labels[0]), activation = 'softmax'))

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit(xtrain, ytrain, epochs=20, batch_size=32, validation_data = (xtest, ytest))

Epoch 1/20
400/400 [==============================] - 3s 5ms/step - loss: 1.3545 - accuracy: 0.4688 - val_loss: 0.8205 - val_accuracy: 0.7119
Epoch 2/20
400/400 [==============================] - 2s 5ms/step - loss: 0.3405 - accuracy: 0.8938 - val_loss: 0.5289 - val_accuracy: 0.8241
Epoch 3/20
400/400 [==============================] - 2s 5ms/step - loss: 0.0583 - accuracy: 0.9852 - val_loss: 0.6057 - val_accuracy: 0.8238
Epoch 4/20
400/400 [==============================] - 2s 5ms/step - loss: 0.0247 - accuracy: 0.9948 - val_loss: 0.6071 - val_accuracy: 0.8259
Epoch 5/20
400/400 [==============================] - 2s 5ms/step - loss: 0.0174 - accuracy: 0.9961 - val_loss: 0.6347 - val_accuracy: 0.8197
Epoch 6/20
400/400 [==============================] - 2s 5ms/step - loss: 0.0153 - accuracy: 0.9967 - val_loss: 0.7111 - val_accuracy: 0.8206
Epoch 7/20
400/400 [==============================] - 2s 5ms/step - loss: 0.0110 - accuracy: 0.9975 - val_loss: 0.7303 - val_accuracy: 0.8175
Epoch 

In [13]:
input_text = "I got a job offer, I'm going to skydive."

In [14]:
#preprocess input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)

prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 70ms/step
['joy']
